# AutoML

In [1]:
from pprint import pprint

from autosklearn.regression import AutoSklearnRegressor

ModuleNotFoundError: No module named 'autosklearn'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [ ]:
raw_data = pd.read_csv('data/train_data_hs.csv',)

In [ ]:
train_data = raw_data[~(raw_data.id%10).isin([3,5,7])]
vali_data = raw_data[(raw_data.id%10).isin([3,5,7])]

In [ ]:
y_col = ['Target']

In [ ]:
x_col = list(raw_data.columns)
x_col.remove('id')
x_col.remove('Target')

In [ ]:
train_X = train_data[x_col]

In [ ]:
train_y = train_data[y_col]

In [ ]:
model = AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,)

In [ ]:
model.fit(train_X, train_y, dataset_name='diabetes')

In [ ]:
pprint(model.show_models(), indent=4)

In [ ]:
test_data = vali_data.copy()

In [ ]:
test_data['pred'] = model.predict(vali_data[x_col])

In [ ]:
plt.figure(figsize = (5,5))
g1 = sns.scatterplot(data=test_data, x='Target', y='pred',)
g1.set_xlim([0, 30])
g1.set_ylim([0, 30])
sns.lineplot(x=[0, 30], y=[0, 30], color='red')

In [ ]:
round(NMAE(test_data.Target, test_data.pred)*100, 3)

|종류|NMAE|
|---|---|
|AutoML|15.766|

# Ensemble

In [ ]:
raw_data = pd.read_csv('data/train_data_hs.csv',)

In [ ]:
raw_data['Target'].hist(bins=50)

In [ ]:
raw_data[raw_data['Target']>15]['Diameter'].hist(bins=50, color='red', alpha=0.5)
raw_data[raw_data['Target']<=15]['Diameter'].hist(bins=50, color='blue', alpha=0.5)
print(raw_data[raw_data['Target']>15]['Diameter'].min())

In [ ]:
y_col = ['Target']

In [ ]:
x_col = list(raw_data.columns)
x_col.remove('id')
x_col.remove('Target')

In [ ]:
train_data = raw_data[~(raw_data.id%10).isin([3,5,7])]
vali_data = raw_data[(raw_data.id%10).isin([3,5,7])]

* md_name == 0  : old
* md_name == 1  : young

In [ ]:
# train_data1 = train_data[train_data['Diameter'] >= 0.315]
# train_data2 = train_data[train_data['Diameter'] < 0.315]

In [ ]:
test_data = vali_data.copy()
res = []

for md_name in [0, 1]:
    
    if md_name == 0:
        train_data2 = train_data[train_data['Diameter'] >= 0.315]
        test_data2 = test_data[test_data['Diameter'] >= 0.315]
    if md_name == 1:
        train_data2 = train_data[train_data['Diameter'] < 0.315]
        test_data2 = test_data[test_data['Diameter'] < 0.315]
        
    train_X = train_data2[x_col]

    train_y = train_data2[y_col]

    model = AutoSklearnRegressor(
        time_left_for_this_task=120,
        per_run_time_limit=30,)

    model.fit(train_X, train_y, dataset_name='diabetes')
    
    res.append(model.predict(test_data2[x_col]))

In [ ]:
res

In [ ]:
test_data.loc[test_data['Diameter'] >= 0.315, 'pred'] = res[0]
test_data.loc[test_data['Diameter'] < 0.315, 'pred'] = res[1]

In [ ]:
plt.figure(figsize = (5,5))
g1 = sns.scatterplot(data=test_data, x='Target', y='pred',)
g1.set_xlim([0, 30])
g1.set_ylim([0, 30])
sns.lineplot(x=[0, 30], y=[0, 30], color='red')

In [ ]:
round(NMAE(test_data.Target, test_data.pred)*100, 3)

# 제출

In [ ]:
test_data = pd.read_csv('data/test.csv')
test_data = test_data.join(pd.get_dummies(test_data['Gender'])).drop(['Gender'], axis=1)
test_data[['F', 'I', 'M']] = test_data[['F', 'I', 'M']].astype('int')

In [ ]:
res = []

for md_name in [0, 1]:
    
    if md_name == 0:
        train_data2 = train_data[train_data['Diameter'] >= 0.315]
        test_data2 = test_data[test_data['Diameter'] >= 0.315]
    if md_name == 1:
        train_data2 = train_data[train_data['Diameter'] < 0.315]
        test_data2 = test_data[test_data['Diameter'] < 0.315]
        
    train_X = train_data2[x_col]

    train_y = train_data2[y_col]

    model = AutoSklearnRegressor(
        time_left_for_this_task=120,
        per_run_time_limit=30,)

    model.fit(train_X, train_y, dataset_name='diabetes')
    
    res.append(model.predict(test_data2[x_col]))

In [ ]:
test_data.loc[test_data['Diameter'] >= 0.315, 'pred'] = res[0]
test_data.loc[test_data['Diameter'] < 0.315, 'pred'] = res[1]

In [ ]:
res_data = test_data[['id', 'pred']]
res_data.rename(columns={'pred':'Target'}, inplace=True)

In [ ]:
res_data.to_csv('result/res_autosklearn_split_diameter0315.csv', index=False)